# fred

> Retrieve and process data from the St. Louis FRED

This module is a (thin) wrapper on the `fred_api` module, which itself is a fork of [fredapi](https://github.com/mortada/fredapi).

To use the functions in the `fred` module, you'll need an API key from the St. Louis FRED. 

Get one [here](https://fred.stlouisfed.org/docs/api/api_key.html) and store it in your environment variables under the name `FRED_API_KEY` 

Alternatively, you can supply the API key directly as the `api_key` parameter in each function in the `fred` module.

In [ ]:
#| default_exp fred.fred

In [ ]:
#| exports
import os, time

import pandas as pd
from finsets.fred.fred_api import Fred

import pandasmore as pdm

In [ ]:
#| export 
def download(series: str=None, # FRED series name
               api_key: str=None # FRED api key. If None, will use os.getenv("FRED_API_KEY")
               ) -> pd.DataFrame: 
    """Retrieves series from FRED and returns it as a pd.DataFrame."""

    if api_key is None: api_key = os.getenv("FRED_API_KEY")
    api = Fred(api_key=api_key)
    return api.get_series(series).to_frame(name=series)

In [ ]:
download('GDP')

,GDP
1946-01-01,NaN
1946-04-01,NaN
1946-07-01,NaN
1946-10-01,NaN
1947-01-01,243.164
...,...
2022-01-01,24740.480
2022-04-01,25248.476
2022-07-01,25723.941
2022-10-01,26137.992


In [ ]:
#| export 
def clean(series: str=None, # FRED series name
               label: str=None, # Name you want to give to the series in the output DataFrame. If None, use lowercase of `series`
               api_key: str=None # FRED api key. If None, will use os.getenv("FRED_API_KEY")
               ) -> pd.DataFrame: 
    """Retrieves series from FRED, cleans the date and sets it as index"""

    if api_key is None: api_key = os.getenv("FRED_API_KEY")
    api = Fred(api_key=api_key)
    freq = api.get_series_info('GDP').frequency_short 
    time.sleep(1)
    if label is None: label = str.lower(series)
    df = download(series).rename(columns={series:label}).reset_index().rename({'index':'date'},axis=1)
    df = pdm.setup_tseries(df, freq=freq).drop('date', axis=1)
    return df 

In [ ]:
clean('GDP', label='Nominal GDP')

,dtdate,Nominal GDP
Qdate,,
1946Q1,1946-01-01,NaN
1946Q2,1946-04-01,NaN
1946Q3,1946-07-01,NaN
1946Q4,1946-10-01,NaN
1947Q1,1947-01-01,243.164
...,...,...
2022Q1,2022-01-01,24740.480
2022Q2,2022-04-01,25248.476
2022Q3,2022-07-01,25723.941


In [ ]:
#| export 
def get_series_info(series: str=None, # FRED series name
                    api_key: str=None # FRED api key. If None, will use os.getenv("FRED_API_KEY")
                    ) -> pd.Series:
    """Get metadata from FRED for given `series` from FRED"""

    if api_key is None: api_key = os.getenv("FRED_API_KEY")
    api = Fred(api_key=api_key)
    return api.get_series_info(series)

In [ ]:
get_series_info('GDP')

id                                                                         GDP
realtime_start                                                      2023-07-14
realtime_end                                                        2023-07-14
title                                                   Gross Domestic Product
observation_start                                                   1947-01-01
observation_end                                                     2023-01-01
frequency                                                            Quarterly
frequency_short                                                              Q
units                                                      Billions of Dollars
units_short                                                          Bil. of $
seasonal_adjustment                            Seasonally Adjusted Annual Rate
seasonal_adjustment_short                                                 SAAR
last_updated                                        

In [ ]:
#| export 
def search(search_text: str=None, # What to search for
              order_by: str='popularity', # How to order search results; try `search_rank` if you don't find what you were looking for
              nr_results: int=10, # How many results to output
              api_key: str=None # FRED api key. If None, will use os.getenv("FRED_API_KEY")
              ) -> pd.DataFrame:
    """Search FRED for a given `search_text`, sort by popularity and return only the first `nr_results`"""

    if api_key is None: api_key = os.getenv("FRED_API_KEY")
    api = Fred(api_key=api_key)  
    return api.search(search_text, order_by=order_by)\
              .pipe(pdm.order_columns, ['title', 'popularity','frequency_short', 'observation_start', 'observation_end'])\
              .iloc[:nr_results]
              

In [ ]:
search("three month treasury bill", order_by='popularity',nr_results=3)

,title,popularity,frequency_short,observation_start,observation_end,id,realtime_start,realtime_end,frequency,units,units_short,seasonal_adjustment,seasonal_adjustment_short,last_updated,notes
series id,,,,,,,,,,,,,,,
TB3MS,"3-Month Treasury Bill Secondary Market Rate, D...",77,M,1934-01-01,2023-06-01,TB3MS,2023-07-14,2023-07-14,Monthly,Percent,%,Not Seasonally Adjusted,NSA,2023-07-03 15:23:09-05:00,"Averages of Business Days, Discount Basis"
DTB3,"3-Month Treasury Bill Secondary Market Rate, D...",74,D,1954-01-04,2023-07-12,DTB3,2023-07-14,2023-07-14,Daily,Percent,%,Not Seasonally Adjusted,NSA,2023-07-13 15:21:13-05:00,Discount Basis
DGS3MO,Market Yield on U.S. Treasury Securities at 3-...,73,D,1981-09-01,2023-07-12,DGS3MO,2023-07-14,2023-07-14,Daily,Percent,%,Not Seasonally Adjusted,NSA,2023-07-13 15:21:10-05:00,For further information regarding treasury con...


In [ ]:
time.sleep(1)
search("three month treasury bill", order_by='search_rank',nr_results=3)

,title,popularity,frequency_short,observation_start,observation_end,id,realtime_start,realtime_end,frequency,units,units_short,seasonal_adjustment,seasonal_adjustment_short,last_updated,notes
series id,,,,,,,,,,,,,,,
TB3MS,"3-Month Treasury Bill Secondary Market Rate, D...",77,M,1934-01-01,2023-06-01,TB3MS,2023-07-14,2023-07-14,Monthly,Percent,%,Not Seasonally Adjusted,NSA,2023-07-03 15:23:09-05:00,"Averages of Business Days, Discount Basis"
DTB3,"3-Month Treasury Bill Secondary Market Rate, D...",74,D,1954-01-04,2023-07-12,DTB3,2023-07-14,2023-07-14,Daily,Percent,%,Not Seasonally Adjusted,NSA,2023-07-13 15:21:13-05:00,Discount Basis
WTB3MS,"3-Month Treasury Bill Secondary Market Rate, D...",40,W,1954-01-08,2023-07-07,WTB3MS,2023-07-14,2023-07-14,"Weekly, Ending Friday",Percent,%,Not Seasonally Adjusted,NSA,2023-07-10 15:17:12-05:00,"Averages of Business Days, Discount Basis"


In [ ]:
#| export
def common_raw_vars() -> pd.DataFrame:
    """List of FRED series commonly used in finance research"""

    vars = [
    ['TB3MS','yield_3mt','3-month treasury bill secondary market rate', 'M', 'NSA',1934],
    ['DTB3','yield_3mt','3-month treasury bill secondary market rate', 'D', 'NSA',1954],
    ['GS10','yield_10yt','10-year treasury constant maturity market yield', 'M', 'NSA',1953],
    ['DGS10','yield_10yt', '10-year treasury constant maturity market yield', 'D', 'NSA',1962],
    ['GS1','yield_1yt', '1-year treasury constant maturity market yield', 'M', 'NSA', 1953],
    ['DGS1','yield_1yt', '1-year treasury constant maturity market yield', 'D', 'NSA', 1962],
    ['AAA','yield_aaa', 'Moodys Aaa seasoned corporate bond yield', 'M', 'NSA', 1919], 
    ['BAA','yield_baa', 'Moodys Baa seasoned corporate bond yield', 'M', 'NSA', 1919], 
    ['DAAA','yield_aaa', 'Moodys Aaa seasoned corporate bond yield', 'D', 'NSA', 1983], 
    ['DBAA','yield_baa', 'Moodys Baa seasoned corporate bond yield', 'D', 'NSA', 1986],     
    ['FEDFUNDS','yield_fedf', 'federal funds effective rate', 'M', 'NSA', 1954], 
    ['DFF','yield_fedf', 'federal funds effective rate', 'D', 'NSA', 1954], 
    ['CPIAUCSL','cpi', 'cpi all urban consumers', 'M', 'SA', 1947],  
    ['CPIAUCNS','cpi', 'cpi all urban consumers', 'M', 'NSA', 1913],  
    ['INDPRO','indprod', 'industrial production', 'M', 'SA', 1919],
    ['IPB50001SQ','indprod', 'industrial production', 'Q', 'SA', 1919],
    ['UNRATE','unemp', 'unemployment rate', 'M', 'SA', 1948], 
    ['GDP','ngdp', 'nominal gdp','Q','SA',1947],
    ['GDPC1','rgdp', 'real gdp','Q','SA',1947],
    ['GNP','ngnp', 'nominal gnp','Q','SA',1947],
    ['GNPC96','rgnp', 'real gnp','Q','SA',1947],
    ['GDPPOT','pot_rgdp','real potential gdp','Q','SA',1949],
    ['USREC','rec_dum', 'nber recession indicator', 'M', 'NSA', 1854], 
    ['RECPROUSM156N','rec_prob', 'nber real time recession probability', 'M', 'NSA', 1967],
    ['CFNAI','cfnai','chicago fed national activity index','M','NSA',1967],
    ['UMCSENT','sent_mich','U Michigan consumer confidence index','M','NSA',1977],
    ['MICH','exp_inflation','U Michigan inflation expectation','M','NSA',1978],
    ['USEPUINDXM','pu_bbd','overall EPU index from Baker,Bloom,Davis','M', 'NSA',1985],
    ['USEPUNEWSINDXM','punews_bbd','news comp of EPU index from BBD','M', 'NSA',1985],
    ['USEPUINDXD','pu_bbd','D EPU from BBD','D','NSA',1985],
    ['VIXCLS','vix','VIX index on snp100 from CBOE 1month','D','NSA',1990],
    ['VXOCLS','vxo','VXO index on snp500 from CBOE','D','NSA',1986]
    ]
    return pd.DataFrame(data = vars, columns = ['id','label','desc','freq','season_adj','starts']) 


In [ ]:
common_raw_vars()

,id,label,desc,freq,season_adj,starts
0,TB3MS,yield_3mt,3-month treasury bill secondary market rate,M,NSA,1934
1,DTB3,yield_3mt,3-month treasury bill secondary market rate,D,NSA,1954
2,GS10,yield_10yt,10-year treasury constant maturity market yield,M,NSA,1953
3,DGS10,yield_10yt,10-year treasury constant maturity market yield,D,NSA,1962
4,GS1,yield_1yt,1-year treasury constant maturity market yield,M,NSA,1953
5,DGS1,yield_1yt,1-year treasury constant maturity market yield,D,NSA,1962
6,AAA,yield_aaa,Moodys Aaa seasoned corporate bond yield,M,NSA,1919
7,BAA,yield_baa,Moodys Baa seasoned corporate bond yield,M,NSA,1919
8,DAAA,yield_aaa,Moodys Aaa seasoned corporate bond yield,D,NSA,1983
9,DBAA,yield_baa,Moodys Baa seasoned corporate bond yield,D,NSA,1986


In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()